In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from statsmodels.formula.api import ols
import statsmodels.api as sm
HS300=pd.read_csv('tudata/HS300.csv').sort_values(by='trade_date')
SZZS = pd.read_csv('tudata/SZZS.csv').sort_values(by='trade_date')
daily = pd.read_csv('tudata/daily_000001.csv').sort_values(by='trade_date')
daily_000156 = pd.read_csv('tudata/daily_000156.csv').sort_values(by='trade_date')
daily_basic_000156 = pd.read_csv('tudata/daily_basic_000156.csv').sort_values(by='trade_date')

In [ ]:
#1描述性统计
#1.1频数分布直方图

sns.distplot(daily['pct_chg'], kde=True,rug=True);

In [ ]:
#1.2数据位置 平均数(mean)，中位数(median)，分为数(quantile)，众数(mode)
print(daily['pct_chg'].mean())#平均数
print(daily['pct_chg'].median())#中位数
print(daily['pct_chg'].quantile(0.25))#下四分位数
print(daily['pct_chg'].mode())#众数

In [ ]:
#1.3数据离散度 极差(range)平均绝对偏差(Mean Absolute Deviation)方差(Variance)标准差(standard deviation)
print(daily['pct_chg'].max()-daily['pct_chg'].min())
print(daily['pct_chg'].mad())
print(daily['pct_chg'].var())
print(daily['pct_chg'].std())

In [ ]:
#2随机变量 
#2.1 连续随机变量

density=stats.kde.gaussian_kde(HS300['pct_chg'])
bins=np.arange(-5,5,0.02)

f, axes = plt.subplots(2, 1)
sns.distplot(HS300['pct_chg'], kde=True,rug=True, ax=axes[0]);
sns.distplot(HS300['pct_chg'].cumsum(), kde=True,rug=True, ax=axes[1]);


In [ ]:
#2.2(离散)二项分布(A binomial discrete random variable) 预测上涨概率
p_rise = len(HS300['pct_chg'][HS300['pct_chg']>0])/len(HS300['pct_chg'])#上涨概率
print(binom.pmf(6,10,p_rise))#10天6天上涨的概率 (pmf=Probability mass function)

In [ ]:
#2.3(连续)正态分布(Normal/Gaussian distribution)
loc=HS300['pct_chg'].mean();scale=HS300['pct_chg'].std();
print(stats.norm.interval(0.95, loc=loc, scale=scale))#概率95%的置信区间
print(stats.norm.pdf(0.5,loc=loc, scale=scale))#收益为0.5的概率
print(stats.norm.cdf(0.5,loc=loc, scale=scale))#收益从负无穷到0.5的概率
print(stats.norm.ppf(0.6688,loc=loc, scale=scale))#累计概率0.6688的对应收益
print(stats.norm.stats(loc=loc, scale=scale,moments='mvsk'))

In [ ]:
#2.4相关性分析 corr,决定系数 corr平方 ,双变量散点图
print(pow(HS300['pct_chg'].corr(daily['pct_chg']),2))
df=pd.concat([daily['trade_date'],daily['pct_chg'], HS300['pct_chg']], keys=['date','GS', 'HS'], axis=1)
sns.jointplot("GS", "HS", df,kind="reg")

In [24]:
#3推断统计 α弃真概率，β取伪概率
#3.1平均收益
print(stats.ttest_1samp(HS300['pct_chg'],0))#p值(平均收益为0的概率)0.2864965907754479大于5%，故平均收益为0
#(独立样本)平安银行和华数传媒的平均收益相等
print(stats.ttest_ind(daily_000156['pct_chg'],daily['pct_chg']))
#(配对样本)沪深300和上证综指的平均收益相等
print(stats.ttest_rel(HS300['pct_chg'],SZZS['pct_chg']))

Ttest_1sampResult(statistic=-1.0684086602799998, pvalue=0.2864965907754479)
Ttest_indResult(statistic=-0.34204171476936973, pvalue=0.73253788683603)
Ttest_relResult(statistic=0.19011826941764062, pvalue=0.8493901824551036)


In [35]:
df=pd.concat([daily_000156['trade_date'],daily_000156['pct_chg'], daily_basic_000156['pe']],
             keys=['date','pct', 'pe'], axis=1)
model = ols('pct ~ C(pe)',data=df).fit()
table1 = sm.stats.anova_lm(model)
print(table1)#PR=0.398586说明有关系

            df      sum_sq   mean_sq         F    PR(>F)
C(pe)     82.0  529.846312  6.461540  1.108654  0.398586
Residual  25.0  145.706902  5.828276       NaN       NaN


In [39]:
model=sm.OLS(HS300['pct_chg'],sm.add_constant(SZZS['pct_chg'].rename("SZZS"))).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                pct_chg   R-squared:                       0.951
Model:                            OLS   Adj. R-squared:                  0.951
Method:                 Least Squares   F-statistic:                     4320.
Date:                Thu, 19 Sep 2019   Prob (F-statistic):          4.81e-147
Time:                        20:49:35   Log-Likelihood:                -48.919
No. Observations:                 223   AIC:                             101.8
Df Residuals:                     221   BIC:                             108.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0099      0.020      0.486      0.6